## Load & Import Libraries

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests


!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 


import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  25.02 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  36.76 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  43.17 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  33.92 MB/s
vincent-0.4.4- 100% |###################

## FourSquare API Credentials

In [2]:
CLIENT_ID = '54N0C55XHKRGUOA5GHJ312NY5WTLQOM022KMMSXH52KDIHGW' # your Foursquare ID
CLIENT_SECRET = 'UOCG424JZ23KFO0LA2U40KVB25IDKVQIJV0YM2XPCKGKZLKJ' # your Foursquare Secret
VERSION = '20190206' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 54N0C55XHKRGUOA5GHJ312NY5WTLQOM022KMMSXH52KDIHGW
CLIENT_SECRET:UOCG424JZ23KFO0LA2U40KVB25IDKVQIJV0YM2XPCKGKZLKJ


## Get Venues From Foursquare

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius = 4000):
    LIMIT = 1000
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return relevant information for nearby venues
        venues_list.append([(name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

## Aggregate Toronto Neighborhoods

In [28]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url, 'lxml')

table = str(soup.find("table", { "class" : "wikitable sortable" }))

df = pd.read_html(table)[0]
df.columns = ['PostalCode', 'Borough', 'Neighborhood']
df.drop(df.index[0], inplace=True)
df = df[df['Borough']!="Not assigned"]
df.Neighborhood[df.Neighborhood=="Not assigned"] = df.Borough[df.Neighborhood=="Not assigned"]
df=df.groupby(['PostalCode']).agg({'Borough': 'first','Neighborhood':', '.join})
df.reset_index(inplace=True)
df.shape

url="http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv"
coordinates=pd.read_csv(url)
coordinates.columns = ['PostalCode', 'Latitude', 'Longitude']
df1 = pd.merge(df,coordinates, on="PostalCode")

df1 = df1[df1['Borough'].str.contains('Toronto')].reset_index(drop=True)
df1.head(10)

,PostalCode,Neighborhood,Borough,Latitude,Longitude
0,M4E,The Beaches,East Toronto,43.676357,-79.293031
1,M4K,"The Danforth West, Riverdale",East Toronto,43.679557,-79.352188
2,M4L,"The Beaches West, India Bazaar",East Toronto,43.668999,-79.315572
3,M4M,Studio District,East Toronto,43.659526,-79.340923
4,M4N,Lawrence Park,Central Toronto,43.728020,-79.388790
5,M4P,Davisville North,Central Toronto,43.712751,-79.390197
6,M4R,North Toronto West,Central Toronto,43.715383,-79.405678
7,M4S,Davisville,Central Toronto,43.704324,-79.388790
8,M4T,"Moore Park, Summerhill East",Central Toronto,43.689574,-79.383160
9,M4V,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Central Toronto,43.686412,-79.400049


## Gather Venues From FourSquare and Use Neighborhood Locations as Inputs

In [5]:
venues = getNearbyVenues(names=df1['Neighborhood'],
                        latitudes=df1['Latitude'],
                        longitudes=df1['Longitude']
                                  )
venues.columns =["Neighborhood","Neighborhood Latitude", "Neighborhood Longitude", "Venue", "Latitude", "Longitude", "Category" ]
venues.head(10)

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Latitude,Longitude,Category
0,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
1,The Beaches,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop
2,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
3,The Beaches,43.676357,-79.293031,Bagels On Fire,43.672864,-79.286784,Bagel Shop
4,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
5,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
6,The Beaches,43.676357,-79.293031,Kew Gardens,43.669038,-79.298538,Park
7,The Beaches,43.676357,-79.293031,I'm On The Beach,43.670364,-79.284696,Beach
8,The Beaches,43.676357,-79.293031,The Remarkable Bean,43.672801,-79.287038,Coffee Shop
9,The Beaches,43.676357,-79.293031,Mastermind Toys,43.671453,-79.293971,Toy / Game Store


## Find Restaurants, Coffee Shops, Hotels and Bars, then Create a Dataframe for Each

In [6]:
coffee_shops = venues[venues['Category']=='Coffee Shop']
restaurants = venues[venues['Category'].str.contains('Restaurant') ==1]
bars = venues[(venues['Category'].str.contains('Bar') ==1) & (venues['Category'].str.contains('Barbershop')==0)]
hotels = venues[(venues['Category'].str.contains('Hotel') ==1) ]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Latitude,Longitude,Category
8,The Beaches,43.676357,-79.293031,The Remarkable Bean,43.672801,-79.287038,Coffee Shop
11,The Beaches,43.676357,-79.293031,Buds Coffee Bar,43.669375,-79.303218,Coffee Shop
54,The Beaches,43.676357,-79.293031,Starbucks,43.668370,-79.308015,Coffee Shop
72,The Beaches,43.676357,-79.293031,Circus Coffee House,43.685483,-79.315364,Coffee Shop
75,The Beaches,43.676357,-79.293031,Starbucks,43.682446,-79.327232,Coffee Shop
121,"The Danforth West, Riverdale",43.679557,-79.352188,Hailed Coffee,43.666900,-79.345432,Coffee Shop
135,"The Danforth West, Riverdale",43.679557,-79.352188,Merchants of Green Coffee,43.659986,-79.354299,Coffee Shop
146,"The Danforth West, Riverdale",43.679557,-79.352188,Te Aro,43.661373,-79.338577,Coffee Shop
148,"The Danforth West, Riverdale",43.679557,-79.352188,Dark Horse Espresso Bar,43.658498,-79.352356,Coffee Shop
163,"The Danforth West, Riverdale",43.679557,-79.352188,Sumach Espresso,43.658135,-79.359515,Coffee Shop


In [7]:
(restaurants.head(10))

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Latitude,Longitude,Category
2,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
14,The Beaches,43.676357,-79.293031,Veloute Bistro,43.672267,-79.289584,French Restaurant
19,The Beaches,43.676357,-79.293031,Budapest Restaurant,43.680946,-79.310110,Hungarian Restaurant
28,The Beaches,43.676357,-79.293031,Udupi Palace,43.672480,-79.321275,Indian Restaurant
29,The Beaches,43.676357,-79.293031,Lake Inez,43.672520,-79.320712,Asian Restaurant
31,The Beaches,43.676357,-79.293031,Jatujak,43.688421,-79.270073,Thai Restaurant
40,The Beaches,43.676357,-79.293031,The Wren,43.682467,-79.328079,American Restaurant
41,The Beaches,43.676357,-79.293031,Xola,43.672603,-79.288080,Mexican Restaurant
43,The Beaches,43.676357,-79.293031,Melanie's Bistro,43.684800,-79.317167,French Restaurant
57,The Beaches,43.676357,-79.293031,Rendez-Vous Restaurant Bar & Cafe,43.682570,-79.327544,Ethiopian Restaurant


## View Bars Dataframe

In [8]:
(bars.head(10))

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Latitude,Longitude,Category
49,The Beaches,43.676357,-79.293031,Relish Bar & Grill,43.686280,-79.310980,Bar
81,The Beaches,43.676357,-79.293031,Pinkerton Snack Bar,43.668900,-79.337309,Cocktail Bar
85,The Beaches,43.676357,-79.293031,The Only Cafe,43.680409,-79.337898,Beer Bar
87,The Beaches,43.676357,-79.293031,The Shore Leave,43.684200,-79.319413,Cocktail Bar
91,The Beaches,43.676357,-79.293031,Hitch Bar,43.663250,-79.330649,Bar
97,The Beaches,43.676357,-79.293031,Sauce,43.682442,-79.328171,Bar
120,"The Danforth West, Riverdale",43.679557,-79.352188,The Only Cafe,43.680409,-79.337898,Beer Bar
128,"The Danforth West, Riverdale",43.679557,-79.352188,Pinkerton Snack Bar,43.668900,-79.337309,Cocktail Bar
158,"The Danforth West, Riverdale",43.679557,-79.352188,The Comrade,43.659346,-79.347932,Bar
188,"The Danforth West, Riverdale",43.679557,-79.352188,Greenhouse Juice Co,43.679101,-79.390686,Juice Bar


## View Hotels Dataframe

In [9]:
(hotels.head(10))

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Latitude,Longitude,Category
138,"The Danforth West, Riverdale",43.679557,-79.352188,The Broadview Hotel,43.659060,-79.350030,Hotel
194,"The Danforth West, Riverdale",43.679557,-79.352188,The Grand Hotel & Suites Toronto,43.656449,-79.374110,Hotel
284,"The Beaches West, India Bazaar",43.668999,-79.315572,The Broadview Hotel,43.659060,-79.350030,Hotel
305,Studio District,43.659526,-79.340923,The Broadview Hotel,43.659060,-79.350030,Hotel
398,Studio District,43.659526,-79.340923,The Grand Hotel & Suites Toronto,43.656449,-79.374110,Hotel
830,"Moore Park, Summerhill East",43.689574,-79.383160,Four Seasons Hotel Toronto,43.671796,-79.389457,Hotel
932,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,Four Seasons Hotel Toronto,43.671796,-79.389457,Hotel
1011,Rosedale,43.679563,-79.377529,Four Seasons Hotel Toronto,43.671796,-79.389457,Hotel
1064,Rosedale,43.679563,-79.377529,The Grand Hotel & Suites Toronto,43.656449,-79.374110,Hotel
1098,Rosedale,43.679563,-79.377529,The Broadview Hotel,43.659060,-79.350030,Hotel


## Coffee Shops Dataframe

In [29]:
coffee_shops.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Latitude,Longitude,Category
8,The Beaches,43.676357,-79.293031,The Remarkable Bean,43.672801,-79.287038,Coffee Shop
11,The Beaches,43.676357,-79.293031,Buds Coffee Bar,43.669375,-79.303218,Coffee Shop
54,The Beaches,43.676357,-79.293031,Starbucks,43.668370,-79.308015,Coffee Shop
72,The Beaches,43.676357,-79.293031,Circus Coffee House,43.685483,-79.315364,Coffee Shop
75,The Beaches,43.676357,-79.293031,Starbucks,43.682446,-79.327232,Coffee Shop
121,"The Danforth West, Riverdale",43.679557,-79.352188,Hailed Coffee,43.666900,-79.345432,Coffee Shop
135,"The Danforth West, Riverdale",43.679557,-79.352188,Merchants of Green Coffee,43.659986,-79.354299,Coffee Shop
146,"The Danforth West, Riverdale",43.679557,-79.352188,Te Aro,43.661373,-79.338577,Coffee Shop
148,"The Danforth West, Riverdale",43.679557,-79.352188,Dark Horse Espresso Bar,43.658498,-79.352356,Coffee Shop
163,"The Danforth West, Riverdale",43.679557,-79.352188,Sumach Espresso,43.658135,-79.359515,Coffee Shop


## Mark Venues

In [10]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, neighborhood in zip(restaurants['Latitude'], restaurants['Longitude'], restaurants['Neighborhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
for lat, lng, neighborhood in zip(coffee_shops['Latitude'], coffee_shops['Longitude'], coffee_shops['Neighborhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
for lat, lng, neighborhood in zip(bars['Latitude'], bars['Longitude'], bars['Neighborhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='black',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
for lat, lng, neighborhood in zip(hotels['Latitude'], hotels['Longitude'], hotels['Neighborhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


## Concatanate the venue locations in a new dataframe, then find the central location and the standard deviation

In [11]:
clients = pd.concat([coffee_shops,bars, restaurants, hotels])
data =clients[['Latitude', 'Longitude']].values
x0 =np.mean(data[:,0])
y0=np.mean(data[:,1])

## Use k-Means Clustering to Define 5 Venue Clusters

In [12]:
# set number of clusters
kclusters = 5
# k-Means Clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(data)

# Check cluster labels generated for each row
kmeans.labels_[0:] 

clients['Labels'] = kmeans.labels_

## Mark Venues on Toronto Map Once More With Different Colours For The Clusters

In [14]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for lat, lng, cluster in zip(clients['Latitude'], clients['Longitude'], clients['Labels']):
    label = cluster
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto

## Function That Returns The Mean Radius of The Clusters

In [15]:
def mean_radius(cl,loc):
    num=0
    sum_r = 0
    for x,y in zip(cl['Latitude'],cl['Longitude']):
        r= np.sqrt(((x-loc[0])*110)**2+((y-loc[1])*110*np.cos(x))**2)
        
        sum_r = sum_r+r
        num=num+1
    mean_r = sum_r/num
   

    return mean_r,num

## Find the Number of Venues and the Mean Radius Before Clustering

In [16]:
cl=clients
loc=[x0,y0]
r,num_all=mean_radius(cl,loc)
print(loc,r, num_all)

[43.664140073956766, -79.392042990832479] 3.52106471404 1416


## Find the Number of Venues and Mean Radius for Each of the Clusters

In [17]:
locations = []
radii = []
num_per_cluster = []
for i in range(kclusters):
    cl = clients[clients['Labels'] == i]
    loc = kmeans.cluster_centers_[i]
    r,num = mean_radius(cl,loc)
    locations.append(loc)
    radii.append(r)
    num_per_cluster.append(num)
locations = np.array(locations)
raddi = np.array(radii)

## Calculate the Clusters' Densities

In [18]:
num_per_cluster

[431, 231, 174, 408, 172]

In [19]:
locations

array([[ 43.65390051, -79.37688846],
       [ 43.70337104, -79.39690812],
       [ 43.66966925, -79.33465418],
       [ 43.653691  , -79.40603255],
       [ 43.65630314, -79.44835502]])

In [20]:
3*np.array(radii)

array([ 3.46832996,  4.47672509,  5.10848621,  3.90313489,  5.77448266])

In [21]:
area = (np.pi*(3*np.array(radii))**2)
area

array([  37.79120051,   62.96087053,   81.98498548,   47.86047371,
        104.75530751])

In [22]:
(np.array(num_per_cluster))/area

array([ 11.40477133,   3.66894546,   2.12233983,   8.52477981,   1.64192158])

In [39]:
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, r, lbl in zip(locations[:,0], locations[:,1], radii, range(kclusters)):
    label = lbl
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=r*5, popup=label, color=rainbow[lbl-1], fill=True, fill_color=rainbow[lbl-1], fill_opacity=0.7, parse_html=False).add_to(map_toronto) 
    print(lat,lng,r)
map_toronto

43.6539005086 -79.3768884597 1.15610998778
43.7033710351 -79.396908123 1.4922416967
43.6696692541 -79.3346541813 1.70282873794
43.6536909971 -79.4060325513 1.30104496211
43.6563031359 -79.4483550231 1.92482755395
